In [1]:
import pandas as pd
import folium
import xyzservices.providers as xyz
MAPBOX_API_KEY = open('../../mapbox_api_key', 'r').read()
from PIL import Image
import os, os.path

In [2]:
path = "../../cropped_marker_images"
imgs = [Image.open(os.path.join(path, f)) for f in os.listdir(path)]

points = [
    ["720 Briar Hill Ave", 43.706843167968444, -79.43343763536379, "https://i.imgur.com/qHeXMDn.jpg", "Your travelling spirit seems to have rubbed off on me", "ca"],
    ["The Big Apple", 44.06580278508552, -77.91405655898555, "https://i.imgur.com/ly25nux.jpg", "Quite a LOVEly roadside attraction", "ca"],
    ["Tea Ceremony", 35.711900, 139.792800, "https://i.imgur.com/W7Po077.jpg", "You can really pull off a kimono!", "japan"],
    ["Tokyo Skytree", 35.709400, 139.808600, "https://i.imgur.com/g5xFE4w.jpg", "I love you more than corn soup", "japan"],
    ["Box Burger", 35.256200, 139.048000, "https://i.imgur.com/jlhhitN.jpg", "This photo is really sweet", "japan"],
    ["Kiyomizu-dera", 34.994600, 135.785400, "https://i.imgur.com/Npa3uYv.jpg", "", "japan"],
    ["Masuyacho", 34.997400, 135.781100, "https://i.imgur.com/RO81Kc8.jpg", "", "japan"],
    ["Hiroshima Dreamination", 34.388200, 132.461200, "https://i.imgur.com/i4RVUMD.jpg", "", "japan"],
    ["Miyajima", 34.279200, 132.325100, "https://i.imgur.com/QnYCUoG.jpg", "", "japan"],
    ["Hahoe Folk Village", 36.538600, 128.517000, "https://i.imgur.com/3tDDosH.jpg", "", "korea_south"],
    ["Third Infiltration Tunnel, DMZ", 37.91671648025547, 126.69829308650752, "https://i.imgur.com/t4y01pG.jpg", "", "korea_south"],
    ["Bukchon Hanok Village", 37.5828025947213, 126.98361926829193, "https://i.imgur.com/X9bGRho.jpg", "", "korea_south"],
    ["The Ex", 43.635400, -79.413600, "https://i.imgur.com/OinvUP5.jpg", "", "canada"],
    ["Eataly", 43.66977878442531, -79.38867336311971, "https://i.imgur.com/NOLqlIJ.jpg", "", "canada"],
    ["SkyDome", 43.640947, -79.389594, "https://i.imgur.com/LIf3OMh.jpg", "", "canada"],
    ["Science Centre", 43.716245366555604, -79.34016622715741, "https://i.imgur.com/ngmTlTA.jpeg", "", "canada"],
]

maybes = [
    ["Belmanor", 44.526464496121, -80.35017367446154, "https://i.imgur.com/OlHUw0d.jpg", "", "canada"],
    ["The Melting Point", 34.862678, -111.785881, "https://i.imgur.com/YaMTx4v.jpg", "", "united_states_of_america"],
    ["The Guggenheim", 40.782936, -73.959124, "https://i.imgur.com/MbSRl11.jpg", "", "united_states_of_america"],
    ["Alpine", 44.51520990123986, -80.35446874749427, "https://i.imgur.com/EY1OCO4.jpg", "", "canada"],
    ["Odawara Station", 35.255500, 139.156000, "https://i.imgur.com/yEGPE79.jpg", "", "japan"],
]


In [3]:
markers = [
    ["720 Briar Hill Ave", 43.706843167968444, -79.43343763536379, "Your travelling spirit seems to have rubbed off on me", "ca"],
    ["The Big Apple", 44.06580278508552, -77.91405655898555, "Quite a LOVEly roadside attraction", "ca"],
    ["Tea Ceremony", 35.711900, 139.792800, "You can really pull off a kimono!", "jp"],
    ["Tokyo Skytree", 35.709400, 139.808600, "I love you more than corn soup", "jp"],
    ["Box Burger", 35.256200, 139.048000, "This photo is really sweet", "jp"],
    ["Kiyomizu-dera", 34.994600, 135.785400, "", "jp"],
    ["Masuyacho", 34.997400, 135.781100, "", "jp"],
    ["Hiroshima Dreamination", 34.388200, 132.461200,"", "jp"],
    ["Miyajima", 34.279200, 132.325100, "", "jp"],
    ["Hahoe Folk Village", 36.538600, 128.517000, "", "kr"],
    ["Third Infiltration Tunnel, DMZ", 37.91671648025547, 126.69829308650752, "", "kr"],
    ["Bukchon Hanok Village", 37.5828025947213, 126.98361926829193, "", "kr"],
    ["The Ex", 43.635400, -79.413600, "", "ca"],
    ["Eataly", 43.66977878442531, -79.38867336311971, "", "ca"],
    ["SkyDome", 43.640947, -79.389594, "", "ca"],
    ["Science Centre", 43.716245366555604, -79.34016622715741, "", "ca"],
    ["Belmanor", 44.526464496121, -80.35017367446154, "", "ca"],
    ["The Melting Point", 34.862678, -111.785881, "", "us"],
    ["The Guggenheim", 40.782936, -73.959124, "", "us"],
    ["Alpine", 44.51520990123986, -80.35446874749427, "", "ca"],
    ["Odawara Station", 35.255500, 139.156000, "", "jp"],
    ["Camp Tamarack", 45.10783959418499, -79.23436423119166, "", "ca"],
    ["Wedding", 43.652157912961776, -79.38499594116591, "", "ca"],
    ["Trinidad", 10.676141, -61.466552, "", "tt"],
    ["Thornbury", 44.5621128359011, -80.46074676342506, '', 'ca'],
    ["Laguna del Mar", 19.319057806598735, -81.38437182493313, '', 'ky'],
    ["Cayman Islands", 19.288455006430883, -81.33059588337466, "To be honest, I couldn't find exactly where this was, but I had to include it because it is my favourite photo of all time", 'ky'],
    ["Hell", 19.379049080697488, -81.4067868330394, '', 'ky'],
    ["Rum Point", 19.37328568406275, -81.27106374346873, '', 'ky'],
    ["Nina", 19.35553364323007, -81.37744972135077, '', 'ky'],
    ["New Ho King", 43.65630591796193, -79.39957491043347, 'I only recognize this restaurant from Kendrick Lamar shouting it out in a Drake diss track 😂', 'ca'],
    ["Toronto Zoo", 43.8189470865355, -79.18735016239854, "", "ca"],
    ["Stingray City", 19.37603806165762, -81.3695549021816, '', 'ky'],
    ["Morritt's Tortuga Club", 19.34610515292206, -81.08935684820545, '', 'ky'],
    ["Galleria", 19.32403558792386, -81.3815533590629, '', 'ky'],
    ["Botanic Park", 19.316028588375264, -81.16788441710654, '', 'ky'],
    ["Havana", 23.135136376685626, -82.35958393907437, '', 'cu'],
    ["Magic Kingdom", 28.418976624031686, -81.58184402785857, '', 'us'],
    ["Animal Kingdom", 28.357631103511977, -81.59065390581534, '', 'us'],
    ["Universal Studios", 28.47941380253163, -81.46861328924686, '', 'us'],
    ["SeaWorld", 28.411218372828507, -81.46131495815239, "", "us"],
    ["Zoey's Birth", 19.289269449274258, -81.38025044166397, "", "ky"],
    ["Aspen", 39.18575978372644, -106.82241261227989, "", "us"],
    ["Atlanta", 33.759421392514305, -84.3838125626199, "", "us"],
    ["Viewmount Park", 43.7071328420153, -79.43789276385512, "", "ca"],
    ["Distillery District", 43.65046856423196, -79.35857561793426, "", "ca"],
    ["African Lion Safari", 43.341093214364044, -80.18009990335581, "", "ca"],
    ["Ontario Place", 43.628293903906055, -79.42149935799233, "", "ca"],
    ["High Park", 43.643467977267136, -79.45716842939947, "", "ca"],
    ["Logan's Birth", 43.769485637032, -79.36271563499781, "", "ca"],
    ["Ottawa", 45.423829705977596, -75.6987944191059, "", "ca"],
    ["Niagara Falls", 43.08910147431332, -79.0722456362308, "", "ca"],
    ["Moscow", 55.753208150235615, 37.62142048699601, "", "ru"],
    ["Dubai", 25.117359406983148, 55.19828973826612, "", "ae"],
    ["Marina Beach", 13.060111786296254, 80.28658085939642, "", "in"],
    ["Hyderabad", 17.390658805068966, 78.47325953859709, "", "in"],
    ["London", 51.503213085369865, -0.11970159313566396, "", "uk"],
    ["Mumbai", 19.07652046224088, 72.86513972500391, "", "in"],
    ["Great Wall of China", 40.43195051082253, 116.57040295242233, "", "cn"],
    ["Shanghai", 31.233572951068002, 121.50556433050812, "", "cn"],
    ["Times Square", 40.758047812691615, -73.98559624765734, "", "us"],
    ["Moore Park", 43.69455220972966, -79.37988624007599, "", "ca"],
    ["KBH", 44.363960885982586, -80.53111470040153, "", "ca"],
    ["Whistler", 50.10722063755311, -122.94440093141303, "", "ca"],
    ["Solelim", 46.37675286388689, -81.04042205042512, "", "ca"],
    ["Centre Island", 43.62048410866985, -79.37335695557802, "", "ca"],
]

In [4]:
for filename in os.listdir('../../images'):
    filename = '../../images/' + filename
    os.rename(filename, filename.replace(' ', '_').replace('.jpeg', '.jpg').lower())
img_list = os.listdir('../../images')

In [5]:
def get_popup(popup):
    location = popup['location'].replace(" ", "_").lower()
    if location + '.jpg' in img_list:
        return f'''<h4 style = "text-align: center; font-family: Futura; font-weight: bold;">{popup["location"]}</h4>'''\
               f'''<img src = "https://github.com/Logan-Daniels/Happy-Mothers-Day/blob/main/images/{location}.jpg?raw=true"'''\
               f'''alt = "{popup['location']}"style="max-width: 300px; max-height: 300px;">'''\
               f'''<p style = "text-align: center; font-family: streetwear; font-size: 14px;">{popup["caption"]}</p>'''
    elif location in img_list:
        images = ''
        imgs = os.listdir('../../images/' + location)
        imgs.sort()
        for image in range(len(imgs)):
             if image != '.DS_Store':
                images += f'''<div class = "image__container"><img src = "https://github.com/Logan-Daniels/Happy-Mothers-Day/blob/main/images/{location}/{imgs[image]}?raw=true" alt = "{popup['location']}, Image #{image}" style = "width: 100%;"></div>'''
        html = '''<style>.slideshow {position: relative; animation: fadeInFromLeft 1.5s ease;} .image__container {position: fixed; top: 0px; left: 0; opacity: 0; transition: opacity 1s ease-in-out;} .image__container.active {opacity: 1;}</style>'''\
              f'''{images}'''\
               '''<script>let slideIndex = 0; showSlides(); function showSlides() {const slides = document.getElementsByClassName('image__container'); for (let i = 0; i < slides.length; i++) {slides[i].classList.remove('active');} slideIndex++; if (slideIndex > slides.length) {slideIndex = 1;} slides[slideIndex - 1].classList.add('active'); setTimeout(showSlides, 3000);}</script>'''
        html = '''<style>.slideshow {position: relative; animation: fadeInFromLeft 1.5s ease;} .image__container {display: flex; align-items: center; justify-content: center; position: absolute; top: 0; left: 0; right: 0; bottom: 0; opacity: 0; transition: opacity 1s ease-in-out;} .image__container.active {opacity: 1;}</style>'''\
          f'''{images}'''\
          '''<script>let slideIndex = 0; showSlides(); function showSlides() {const slides = document.getElementsByClassName('image__container'); for (let i = 0; i < slides.length; i++) {slides[i].classList.remove('active');} slideIndex++; if (slideIndex > slides.length) {slideIndex = 1;} slides[slideIndex - 1].classList.add('active'); setTimeout(showSlides, 3000);}</script>'''
        with open(f"../../slideshows/{location}.html", "w") as file:
            file.write(html)
        return f'''<h4 style = "text-align: center; font-family: Futura; font-weight: bold;">{popup["location"]}</h4>'''\
               f'''<iframe src="../../slideshows/{location}.html" height = "250 px"></iframe>'''\
               f'''<p style = "text-align: center; font-family: streetwear; font-size: 14px;">{popup["caption"]}</p>'''
    else:
        raise FileNotFoundError(f"Requested image: {popup['location']} not in directory: images")

                
df = pd.DataFrame(markers, columns = ['location', 'latitude', 'longitude', "caption", "country"])
df['hover_info'] = df.apply(get_popup, axis = 1)
df = df.drop(columns = ['location', 'caption'])

In [6]:
TORONTO_COORDINATES = (43.66977878442531, -79.38867336311971)
m = folium.Map(location = TORONTO_COORDINATES, 
               zoom_start = 12,
               min_zoom = 2,
               max_bounds=True)

tile_provider = xyz.MapBox
tile_provider['accessToken'] = MAPBOX_API_KEY
tile_provider['url'] = 'https://api.mapbox.com/styles/v1/logandaniels/cluryho1d01wt01pbdqtxdp21/tiles/{z}/{x}/{y}{r}?access_token=' + MAPBOX_API_KEY
tile_provider['attribution'] = ' '
tile_provider['html_attribution'] = ' '
tile_provider['min_zoom'] = '2'

folium.TileLayer(
    tiles = tile_provider.build_url(api_key = MAPBOX_API_KEY),
    attr = tile_provider['attribution'],
    name = tile_provider['name'],
    max_zoom = tile_provider['max_zoom'],
    detect_retina = True,
    min_zoom = tile_provider['min_zoom']
).add_to(m)

headers = {'User-Agent': 'Mozilla/5.0'}

for marker in range(len(df)):
    url = "https://raw.githubusercontent.com/Logan-Daniels/Happy-Mothers-Day/main/cropped_marker_images/{}.png".format
    url = url(df['country'].iloc[marker])
    icon = folium.CustomIcon(
        url,
        icon_size = (62, 104),
        icon_anchor = (31, 104),
        popup_anchor = (0, -100),
    )

    folium.Marker(
        location = [df['latitude'].iloc[marker], df['longitude'].iloc[marker]], 
        icon = icon, 
        popup = df['hover_info'].iloc[marker]
    ).add_to(m)
m

In [7]:
m.save('mothers_day.html')